<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MambaGC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# https://github.com/state-spaces/mamba/tree/main

#!git clone https://github.com/state-spaces/mamba.git
#%cd /content/mamba/
#!pip install .

from mamba_ssm import Mamba
import torch

batch, length, dim = 2, 64, 16
x = torch.randn(batch, length, dim).to("cuda")
model = Mamba(
    # This module uses roughly 3 * expand * d_model^2 parameters
    d_model=dim, # Model dimension d_model
    d_state=16,  # SSM state expansion factor
    d_conv=4,    # Local convolution width
    expand=2,    # Block expansion factor
).to("cuda")
y = model(x)
assert y.shape == x.shape

print()
#print(f"{x.shape}")
print(f"{y.shape}")
print()

#!git clone https://github.com/EleutherAI/lm-evaluation-harness
#%cd lm-evaluation-harness
#!pip install -e .

## CUDA Resources insttallation to avoid AssertionError: libcuda.so cannot found!
#!pip install --quiet --upgrade --pre --no-build-isolation torch==2.1.0 torchaudio --index-url https://download.pytorch.org/whl/nightly/cu121


#!nvidia-smi

print()

#!ls /usr/local/cuda/lib64/stubs/libcuda.so
#!rm -rf /lib/x86_64-linux-gnu/libcuda.so
#!rm -rf /lib/x86_64-linux-gnu/libcuda.so.1

#!ln -s /usr/local/cuda/lib64/stubs/libcuda.so /lib/x86_64-linux-gnu/libcuda.so

print()
print('Evaluations')
!python /content/mamba/evals/lm_harness_eval.py --model mamba --model_args pretrained=state-spaces/mamba-130m --tasks lambada_openai,hellaswag,piqa,arc_easy,arc_challenge,winogrande --device cuda --batch_size 64
print()


#!ldconfig -p

print()
print('Inference')
!python /content/mamba/benchmarks/benchmark_generation_mamba_simple.py --model-name "state-spaces/mamba-2.8b" --prompt "My cat wrote all this CUDA code for a new language model and" --topp 0.9 --temperature 0.5
print()


###python benchmarks/benchmark_generation_mamba_simple.py --model-name "EleutherAI/pythia-2.8b" --prompt "My cat wrote all this CUDA code for a new language model and" --topp 0.9 --temperature 0.5
###python evals/lm_harness_eval.py --model hf --model_args pretrained=EleutherAI/pythia-160m --tasks lambada_openai,hellaswag,piqa,arc_easy,arc_challenge,winogrande --device cuda --batch_size 64


torch.Size([2, 64, 16])



2023-12-08:07:05:07,666 INFO     [utils.py:160] NumExpr defaulting to 8 threads.
2023-12-08 07:05:08.451780: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-08 07:05:08.451842: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-08 07:05:08.451875: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-08 07:05:09.535127: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-12-08:07:05:11,906 INFO     [__main__.py:132] Verbosity set to INFO
2023-12-08:07:05:20,058 INFO     [__main__.py:205] Selec